In [179]:
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
import re 
import pyautogui
import pandas as pd

In [180]:
# Создаем таблицу рецептов и таблицу ингредиентов. Только на первом проходе. 
# Потом они хранится где-то и пополняются уникальными рецептами и ингредиентами
# ingr_df = pd.DataFrame(columns=['ingr_id', 'ingr_name'])
# recipe_df = pd.DataFrame(columns=['recipe_name', 'ingredients', 'quantity'])
recipe_df = pd.read_csv('./recipe_df.csv')
ingr_df = pd.read_csv('./ingr_df.csv')

In [181]:
def add_ingr(ingr_name, ingr_df):
    """
    Функция добавляет ингредиент с уникальным именем в таблицу ингредиентов

        Args:
            ingr_name (string): наименование ингредиента
            ingr_df (dataframe): таблица ингредиентов
 
        Returns:
            ingr_df (dataframe): таблица ингредиентов с добавленным уникальным ингредиентом    
    """
    # dataframe текущего ингредиента
    curr_ingr = pd.DataFrame(columns=['ingr_id', 'ingr_name'], index=range(1))
    curr_ingr.ingr_name = ingr_name # наименование текущего ингредиента
    curr_ingr.ingr_id = ingr_df.ingr_id.count() # id текущего ингредиента
    # присоединение текущего ингредиента к исходному dataframe 
    ingr_df = pd.concat([ingr_df, curr_ingr], ignore_index=True)
    # удаление ингредиента, если такое наименование уже было в исходном dataframe  
    ingr_df.drop_duplicates(subset=['ingr_name'], inplace=True)
    return ingr_df

In [182]:
url = pyautogui.prompt(text="Выберите рецепт на сайте https://www.russianfood.com и вставьте его адрес сюда", 
                                    title='Выбор рецепта' , 
                                    default='https://www.russianfood.com/recipes/recipe.php?rid=126032')

In [183]:
# Бефстроганов классический - в разделителях встречается тире
# https://www.russianfood.com/recipes/recipe.php?rid=116909

In [184]:
# Выполняем GET-запрос, содержимое ответа присваивается переменной response
response = requests.get(url, headers={
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер

In [185]:
# Название рецепта
recipe_name = (' ').join(page.title.text.split(' ')[1:-2])

In [186]:
page_recipe = page.find_all('span', class_="")
ingr_dict = {}
for i in range(len(page_recipe)):
    line = str(page_recipe[i].string)
    isCyrillic = bool(re.search('[а-яА-Я]', line))
    if isCyrillic:
        k, v = re.split(' [^а-яА-Я0-9] ', line, maxsplit=1)
#        k, v = re.split(' \W ', line, maxsplit=1) #этот вариант исключает и нижнее почеркивание
        ingr_dict[k] = v
        ingr_df = add_ingr(k, ingr_df)
ingr_dict    

{'Вода (или бульон)': '2-2,5 л',
 'Свекла': '1 шт.',
 'Фасоль': '0,5 стакана',
 'Капуста белокочанная': '200-300 г',
 'Лук репчатый': '1 шт.',
 'Морковь': '1 шт.',
 'Картофель': '2-3 шт.',
 'Корень сельдерея': '50 г',
 'Корень петрушки': '50 г',
 'Чернослив': '6-8 шт.',
 'Томатная паста': '1-2 ст. ложки',
 'Перец черный молотый': 'по вкусу',
 'Соль': 'по вкусу',
 'Перец сладкий': '0,5-1 шт.',
 'Сахар': '1 ч. ложка',
 'Лавровый лист': '1 шт.',
 'Зелень': 'по вкусу',
 'Растительное масло': '6 ст. ложек'}

In [187]:
# Текущий рецепт
recipe = pd.DataFrame(columns=['recipe_name', 'ingredients', 'quantity'])
recipe.ingredients = pd.Series(ingr_dict.keys())
recipe.quantity = pd.Series(ingr_dict.values())
recipe.recipe_name = recipe_name

In [188]:
# присоединение текущего рецепта к исходному dataframe
recipe_df = pd.concat([recipe_df, recipe], ignore_index=True)
# удаление дубликатов рецептов 
recipe_df.drop_duplicates(inplace=True)

In [189]:
recipe_df.to_csv('./recipe_df.csv', index=False)
ingr_df.to_csv('./ingr_df.csv', index=False)

In [190]:
recipe_df

,recipe_name,ingredients,quantity
0,Бефстроганов классический,"Мясо говядины (филейная часть, кострец)",500 г
1,Бефстроганов классический,Лук репчатый крупный,1 шт.
2,Бефстроганов классический,Мука,2 ч. ложки
3,Бефстроганов классический,Сметана,2-3 полных ст. ложки
4,Бефстроганов классический,Масло растительное,для жарки
5,Бефстроганов классический,Зелень укропа,по вкусу
6,Красный борщ с фасолью и черносливом,Вода (или бульон),"2-2,5 л"
7,Красный борщ с фасолью и черносливом,Свекла,1 шт.
8,Красный борщ с фасолью и черносливом,Фасоль,"0,5 стакана"
9,Красный борщ с фасолью и черносливом,Капуста белокочанная,200-300 г


In [191]:
ingr_df

,ingr_id,ingr_name
0,0,Вода (или бульон)
1,1,Свекла
2,2,Фасоль
3,3,Капуста белокочанная
4,4,Лук репчатый
5,5,Морковь
6,6,Картофель
7,7,Корень сельдерея
8,8,Корень петрушки
9,9,Чернослив
